## Load Library

In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering

import warnings

warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
%matplotlib inline

## Load Data

In [2]:
input_file ='../data/results_3k_2hop.json'

# with open(input_file) as f:
#     repo_data = json.load(f,strict=False)
#  JSONDecodeError: Extra data: line 2 column 1 (char 969)
    
file = open(input_file, 'r', encoding='utf-8')
repo_data = {}
for line in file.readlines():
    dic = json.loads(line)
    repo_data = {**repo_data,**dic}
    
print(f'Repository numbers: {len(repo_data)}')
print(f'''https://github.com/natcap/natgeo-dams': 
{repo_data['https://github.com/natcap/natgeo-dams']}''')

Repository numbers: 2960
https://github.com/natcap/natgeo-dams': 
['pypng', 'requests', 'alabaster', 'codecov', 'detox', 'docutils', 'flake8', 'httpbin', 'more-itertools', 'pysocks', 'pytest', 'pytest-cov', 'pytest-httpbin', 'pytest-mock', 'pytest-xdist', 'readme-renderer', 'sphinx', 'tox', 'apipkg', 'appdirs', 'atomicwrites', 'attrs', 'babel', 'bleach', 'blinker', 'brotlipy', 'certifi', 'cffi', 'chardet', 'click', 'configparser', 'contextlib2', 'coverage', 'decorator', 'distlib', 'dnspython', 'entrypoints', 'enum34', 'eventlet', 'execnet', 'filelock', 'flask', 'funcsigs', 'functools32', 'greenlet', 'idna', 'imagesize', 'importlib-metadata', 'importlib-resources', 'itsdangerous', 'jinja2', 'markupsafe', 'mccabe', 'mock', 'monotonic', 'pathlib2', 'pluggy', 'py', 'pycodestyle', 'pycparser', 'pyflakes', 'pygments', 'pytest-forked', 'pytz', 'raven', 'scandir', 'singledispatch', 'six', 'snowballstemmer', 'toml', 'typing', 'urllib3', 'virtualenv', 'webencodings', 'werkzeug', 'zipp']


###  problem: why 2960??

## Process Data
1. repository name -> index
2. dependency name -> index
3. build dep matrix  matrix shape (2960, 28477)

In [3]:
## id->name name->id dictionary
rep_name2index = {}
dep_name2index = {}
for rep,deps in repo_data.items():
    rep_name2index.setdefault(rep,len(rep_name2index))
    for dep in deps:
        dep_name2index.setdefault(dep,len(dep_name2index))

rep_index2name = {}
dep_index2name = {}
for k,v in rep_name2index.items():
    rep_index2name[v] = k

for k,v in dep_name2index.items():
    dep_index2name[v] = k

rep_num = len(rep_name2index)
dep_num = len(dep_name2index)
    
# build matrix
repo_mat = np.zeros((rep_num,dep_num))  # (3012, 15663)
for rep in repo_data:
    rep_index = rep_name2index[rep] # row number
    for dep in repo_data[rep]:
        dep_index =dep_name2index[dep] # col number
        repo_mat[rep_index][dep_index] = 1

## Clustering

In [4]:
##  get the cosine similarity matrix
cos_sim_mat = cosine_similarity(repo_mat)
cos_sim_mat.shape, cos_sim_mat

((2960, 2960),
 array([[1.        , 0.09410436, 0.        , ..., 0.13905533, 0.14509525,
         0.12681432],
        [0.09410436, 1.        , 0.03566882, ..., 0.32928732, 0.22558942,
         0.31905175],
        [0.        , 0.03566882, 1.        , ..., 0.09325048, 0.15811388,
         0.10050378],
        ...,
        [0.13905533, 0.32928732, 0.09325048, ..., 1.        , 0.58976782,
         0.77787815],
        [0.14509525, 0.22558942, 0.15811388, ..., 0.58976782, 1.        ,
         0.38138504],
        [0.12681432, 0.31905175, 0.10050378, ..., 0.77787815, 0.38138504,
         1.        ]]))

In [5]:
repo_mat.shape,repo_mat

((2960, 28477), array([[1., 1., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]))

###  1. Kmeans with repo_mat

In [6]:
%%time

clusters = list(range(2,15))
c_h_scores,s_scores,d_scores = [],[],[]
for k in clusters:
    y_pred = KMeans(n_clusters=k, init='k-means++', max_iter=500).fit_predict(repo_mat)
    c_h_score= metrics.calinski_harabasz_score(repo_mat, y_pred) # maximize 
    s_score = metrics.silhouette_score(repo_mat, y_pred)         # [-1,1]  want it close to 1
    d_score = metrics.davies_bouldin_score(repo_mat, y_pred)     # minimize  want it close to 0
    c_h_scores.append(c_h_score)
    s_scores.append(s_score)
    d_scores.append(d_score)
    print("n_clusters=", k,"score1:",c_h_score,"score2:",s_score,"score3:",d_score)

n_clusters= 2 score1: 1037.3159999545803 score2: 0.6070374259660937 score3: 1.251053509803207
n_clusters= 3 score1: 841.1096022571222 score2: 0.4160917525672647 score3: 1.7806583711237343
n_clusters= 4 score1: 653.7732783597684 score2: 0.36690552381107616 score3: 2.1391056538890036
n_clusters= 5 score1: 559.1849565304647 score2: 0.2254839793143711 score3: 2.109862162641722
n_clusters= 6 score1: 497.1392400412659 score2: 0.2319295499984247 score3: 2.0232958701516783
n_clusters= 7 score1: 435.48959126682246 score2: 0.23709255104064475 score3: 2.3371219940763734
n_clusters= 8 score1: 405.2303178440135 score2: 0.23725043940058713 score3: 1.9305873089305505
n_clusters= 9 score1: 385.1835284745038 score2: 0.24391255129807698 score3: 2.120665720686558
n_clusters= 10 score1: 351.16809868415584 score2: 0.2447501547830813 score3: 2.3145950889453846
n_clusters= 11 score1: 343.93481880549064 score2: 0.25114657361615395 score3: 2.01906783111127
n_clusters= 12 score1: 319.8157083750234 score2: 0.251

#### => result clusetr = 4 OR 5 

###  2. Kmeans + PCA with repo_mat

In [7]:
%%time
pca = PCA(0.95)  
X_pca = pca.fit_transform(repo_mat)
print(X_pca.shape)

clusters = list(range(2,15))
c_h_scores,s_scores,d_scores = [],[],[]
for k in clusters:
    y_pred = KMeans(n_clusters=k, init='k-means++', max_iter=500).fit_predict(X_pca)
    c_h_score= metrics.calinski_harabasz_score(X_pca, y_pred) # maximize 
    s_score = metrics.silhouette_score(X_pca, y_pred)         # [-1,1]  want it close to 1
    d_score = metrics.davies_bouldin_score(X_pca, y_pred)     # minimize  want it close to 0
    c_h_scores.append(c_h_score)
    s_scores.append(s_score)
    d_scores.append(d_score)
    print("n_clusters=", k,"score1:",c_h_score,"score2:",s_score,"score3:",d_score)

(2960, 310)
n_clusters= 2 score1: 1112.3081396235818 score2: 0.620014450723952 score3: 1.2342280724704169
n_clusters= 3 score1: 912.377774457533 score2: 0.44270637289436143 score3: 1.7045305910820243
n_clusters= 4 score1: 712.728829555541 score2: 0.3991274915910202 score3: 2.0417093372179207
n_clusters= 5 score1: 612.8581025609484 score2: 0.2693652068583596 score3: 1.9845713293023717
n_clusters= 6 score1: 525.6504825990539 score2: 0.3554437416632452 score3: 1.8216189504981894
n_clusters= 7 score1: 504.37443524648705 score2: 0.29643168243450957 score3: 1.7339594924246364
n_clusters= 8 score1: 464.6703213283977 score2: 0.29443069405614225 score3: 1.7061391906628356
n_clusters= 9 score1: 433.24083829049187 score2: 0.30181942833300773 score3: 1.6364929197297635
n_clusters= 10 score1: 404.94865985750255 score2: 0.3119710052243168 score3: 1.6028715990913702
n_clusters= 11 score1: 370.42562919109236 score2: 0.31235508920595745 score3: 2.1396585358924045
n_clusters= 12 score1: 352.504783365949

#### => result clusetr = 3 OR 4

###  3. Kmeans with cos_mat  ********

In [8]:
%%time
clusters = list(range(2,15))
c_h_scores,s_scores,d_scores = [],[],[]
for k in clusters:
    y_pred = KMeans(n_clusters=k, init='k-means++', max_iter=500).fit_predict(cos_sim_mat)
    c_h_score= metrics.calinski_harabasz_score(cos_sim_mat, y_pred) # maximize 
    s_score = metrics.silhouette_score(cos_sim_mat, y_pred)         # [-1,1]  want it close to 1
    d_score = metrics.davies_bouldin_score(cos_sim_mat, y_pred)     # minimize  want it close to 0
    c_h_scores.append(c_h_score)
    s_scores.append(s_score)
    d_scores.append(d_score)
    print("n_clusters=", k,"score1:",c_h_score,"score2:",s_score,"score3:",d_score)

n_clusters= 2 score1: 1629.658963462905 score2: 0.32155758854240263 score3: 1.2960948753280037
n_clusters= 3 score1: 1692.4366604781324 score2: 0.35803257103695063 score3: 1.1617266421254182
n_clusters= 4 score1: 1632.3930296179315 score2: 0.35715519241542953 score3: 1.1503481260055015
n_clusters= 5 score1: 1763.9200344421756 score2: 0.3979603191077796 score3: 1.076232339775157
n_clusters= 6 score1: 1843.5137294091924 score2: 0.41969831612185665 score3: 1.0499979575538667
n_clusters= 7 score1: 1774.8402002522869 score2: 0.4111773832812685 score3: 1.0004208468798979
n_clusters= 8 score1: 1765.398089930172 score2: 0.4260975608242277 score3: 1.1123432311270889
n_clusters= 9 score1: 1733.3807404479414 score2: 0.44269632624527694 score3: 1.0290974581985204
n_clusters= 10 score1: 1735.7800054825043 score2: 0.43167897610613193 score3: 1.1180928121090261
n_clusters= 11 score1: 1767.0195878607033 score2: 0.4503430791468564 score3: 1.0000389655214332
n_clusters= 12 score1: 1749.3140238080184 sco

#### => result clusetr = 6

### Spectral Cluster   ********

The edge weight value between two points farther apart is lower, while the edge weight value between two points closer together is higher.

Strength:<br>
1) Spectral clustering only needs the similarity matrix between data, so it is very effective for processing sparse data clustering. This is difficult for traditional clustering algorithms such as K-Means.<br>
2) Due to the use of dimensionality reduction, the complexity of processing high-dimensional data clustering is better than traditional clustering algorithms.

Weakness:<br>
1) If the dimensionality of the final cluster is very high, the running speed of spectral clustering and the final clustering effect are not good due to insufficient dimensionality reduction.<br>
2) The clustering effect depends on the similarity matrix, and the final clustering effect obtained by different similarity matrices may be very different.

In [9]:
%%time
clusters = list(range(2,15))
c_h_scores,s_scores,d_scores = [],[],[]
for k in clusters:
    y_pred = SpectralClustering(n_clusters=k,affinity='precomputed').fit_predict(cos_sim_mat)
    c_h_score= metrics.calinski_harabasz_score(cos_sim_mat, y_pred) # maximize 
    s_score = metrics.silhouette_score(cos_sim_mat, y_pred)         # [-1,1]  want it close to 1
    d_score = metrics.davies_bouldin_score(cos_sim_mat, y_pred)     # minimize  want it close to 0
    c_h_scores.append(c_h_score)
    s_scores.append(s_score)
    d_scores.append(d_score)
    print("n_clusters=", k,"score1:",c_h_score,"score2:",s_score,"score3:",d_score)
    
#Warning:
#/opt/anaconda3/envs/Knowledge_Graph/lib/python3.6/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
#  warnings.warn("Graph is not fully connected, spectral embedding"
#'''

n_clusters= 2 score1: 2.1095054126192703 score2: -0.07703093055985101 score3: 0.9848325428164458
n_clusters= 3 score1: 2.1211496924237525 score2: -0.07597227821824325 score3: 0.9708633005735064
n_clusters= 4 score1: 93.44242476492062 score2: -0.0056383942975032145 score3: 0.977104150063664
n_clusters= 5 score1: 71.54683104365813 score2: -0.003804104891033956 score3: 1.0026351814371548
n_clusters= 6 score1: 2.1286257154830097 score2: -0.07471639884044708 score3: 0.9749437623498939
n_clusters= 7 score1: 2.1326751800987522 score2: -0.07343198015488889 score3: 0.9695599089249434
n_clusters= 8 score1: 4.792972963760513 score2: -0.06724892879536429 score3: 0.9903676183317569
n_clusters= 9 score1: 40.09501655804959 score2: 0.002522054772243455 score3: 1.0248798178735303
n_clusters= 10 score1: 34.09603670991329 score2: 0.0006701453348638753 score3: 0.9936409392536092
n_clusters= 11 score1: 35.01975933067803 score2: 0.01019038572630813 score3: 1.0363033332447555
n_clusters= 12 score1: 5.4724192

#### => result clusetr = 9 !!!


### HDBSCAN Cluster 

strength:
1. Compared with K-Means, HDBSCAN does not need to declare the number of clusters in advance.
2. It can cluster dense data sets of any shape. In contrast, clustering algorithms such as K-Means are generally only suitable for convex data sets.
3. Abnormal points can be found while clustering, and it is not sensitive to abnormal points in the data set.
4. The clustering results are not biased. In contrast, the initial values of clustering algorithms such as K-Means have a great influence on the clustering results

In [10]:
%%time
import hdbscan
import numpy as np

#params 
min_cluster_sizes = [5,10,15]
min_sampless = [10,20,30,40]
cluster_selection_epsilons = [0.1,0.3]
alphas = [0.25,0.5,0.75]


for min_cluster_size in min_cluster_sizes:
    for min_samples in min_sampless:
        for cluster_selection_epsilon in cluster_selection_epsilons:
            for alpha in alphas:
                print("min_cluster_size:",min_cluster_size,'min_samples:',min_samples,
                      "cluster_selection_epsilon:",cluster_selection_epsilon,'alpha:',alpha)

                cluster = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,min_samples=min_samples,
                                          cluster_selection_epsilon=cluster_selection_epsilon,alpha=alpha)
                y_pred= cluster.fit_predict(cos_sim_mat)
                labels = cluster.labels_

                c_h_score= metrics.calinski_harabasz_score(cos_sim_mat, y_pred) # maximize 
                s_score = metrics.silhouette_score(cos_sim_mat, y_pred)         # [-1,1]  want it close 1
                d_score = metrics.davies_bouldin_score(cos_sim_mat, y_pred)     # minimize 
                y_unique = np.unique(labels)
                n_clusters = y_unique.size - (1 if -1 in y_unique else 0)
                print("clusters:",n_clusters ,"score1:",c_h_score,"score2:",s_score,"score3:",d_score)
                print('='*50)

min_cluster_size: 5 min_samples: 10 cluster_selection_epsilon: 0.1 alpha: 0.25
clusters: 149 score1: 59.574034951321025 score2: 0.10791834765770884 score3: 1.2338591657479963
min_cluster_size: 5 min_samples: 10 cluster_selection_epsilon: 0.1 alpha: 0.5
clusters: 146 score1: 61.67183050521367 score2: 0.11176032329721709 score3: 1.2312048721914937
min_cluster_size: 5 min_samples: 10 cluster_selection_epsilon: 0.1 alpha: 0.75
clusters: 103 score1: 101.32949033218966 score2: 0.13416297540106664 score3: 1.235639240643247
min_cluster_size: 5 min_samples: 10 cluster_selection_epsilon: 0.3 alpha: 0.25
clusters: 149 score1: 59.574034951321025 score2: 0.10791834765770884 score3: 1.2338591657479963
min_cluster_size: 5 min_samples: 10 cluster_selection_epsilon: 0.3 alpha: 0.5
clusters: 145 score1: 65.8343781807023 score2: 0.12254737244729981 score3: 1.2307385018768338
min_cluster_size: 5 min_samples: 10 cluster_selection_epsilon: 0.3 alpha: 0.75
clusters: 103 score1: 101.32949033218966 score2: 0.1

clusters: 54 score1: 242.31636942458454 score2: 0.21132672931152646 score3: 1.2565356156093215
min_cluster_size: 10 min_samples: 30 cluster_selection_epsilon: 0.1 alpha: 0.75
clusters: 25 score1: 466.18203344591717 score2: 0.20784713001964988 score3: 1.127441738507059
min_cluster_size: 10 min_samples: 30 cluster_selection_epsilon: 0.3 alpha: 0.25
clusters: 64 score1: 174.87601734710645 score2: 0.16010451156258104 score3: 1.2025357902162157
min_cluster_size: 10 min_samples: 30 cluster_selection_epsilon: 0.3 alpha: 0.5
clusters: 54 score1: 242.31636942458454 score2: 0.21132672931152646 score3: 1.2565356156093215
min_cluster_size: 10 min_samples: 30 cluster_selection_epsilon: 0.3 alpha: 0.75
clusters: 25 score1: 466.18203344591717 score2: 0.20784713001964988 score3: 1.127441738507059
min_cluster_size: 10 min_samples: 40 cluster_selection_epsilon: 0.1 alpha: 0.25
clusters: 63 score1: 194.00216276998296 score2: 0.19061669284107674 score3: 1.214220138415802
min_cluster_size: 10 min_samples: 

min_cluster_size: 15 min_samples: 40 cluster_selection_epsilon: 0.1 alpha: 0.75
clusters: 19 score1: 572.6882498665217 score2: 0.26683735532023717 score3: 1.1006864744530893
                